In [ ]:
import os
import json
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
# Load metadata file
meta_path = 'PKLot/train/_annotations.coco.json'
with open(meta_path) as file:
    meta_train = json.load(file)


In [ ]:
# Create DataFrame from the metadata
train = pd.DataFrame(meta_train['images'])
df_meta = pd.DataFrame(meta_train['annotations'])
train = pd.DataFrame.merge(train,df_meta, how='left', left_on='id', right_on='image_id')
train = train.dropna()
train['id_y'] = train['id_y'].values.astype(int)
train['category_id'] = train['category_id'].values.astype(int)
train['area'] = train['area'].values.astype(int)
train['iscrowd'] = train['iscrowd'].values.astype(int)

In [ ]:
train_img_folder = 'PKLot/grey_train'
image = {}
for img in train['file_name'].unique():
    img_path = os.path.join(train_img_folder, img)
    pixels = np.array(Image.open(img_path).convert('L'))
    pixels = pixels.reshape(640, 640)
    image[img] = pixels
images = np.array(image)

In [ ]:
bboxes_df = train[['file_name', 'bbox']].copy()
bboxes_df = bboxes_df.groupby('file_name', as_index=True).agg({'bbox': list})
bboxes = np.array(bboxes_df['bbox'])

In [ ]:
images = images.astype('float32')
images = images/255
images = images.reshape((len(images),640,640,1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense

model = Sequential()
model.add(Input(shape=(640,640,1)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(images, bboxes)
                    #, validation_data=(images_test, bboxes_test) batch_size=128, epochs=10, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy']) 
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.subplot(1,2,2)
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()